In [3]:
%%capture
import sys, os
import torch, math, os
import sys
sys.path.append("..")

IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    print("Running in Colab!")
    from google.colab import drive

    drive.mount('/content/drive', force_remount=False)
else:
    print("Not running in Colab.")

def resolve_path_gdrive(relativePath):
    if os.path.exists('/content/drive'):
        return '/content/drive/MyDrive/work/gdrive-workspaces/git/nn_catalyst/' + relativePath
    else:
        from utils import get_project_root
        return get_project_root() + "../" + relativePath

print(f"Root project folder is at {resolve_path_gdrive('.')}")

CHECKPOINTS_FOLDER_BASE = "/checkpoints/stn_r3_f849_tlast29"
CHECKPOINTS_FOLDER = resolve_path_gdrive(CHECKPOINTS_FOLDER_BASE) #f'd:/temp{CHECKPOINTS_FOLDER_BASE}'
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.set_float32_matmul_precision("medium")  # to make lightning happy

In [4]:

import pandas as pd
import torch
import numpy as np
from sklearn.preprocessing import StandardScaler
from torch import nn
import torch.nn.functional as F

# Load the data
df = pd.read_csv('few_merged_data_f849_tlast29_reordered_byR2.csv')
X = df.iloc[:, :849]  # First 849 columns are features
y = df.iloc[:, 849:]  # Last 29 columns are targets
from pl.model_impl import *

In [16]:
print(CHECKPOINTS_FOLDER)
from utils import get_project_root
get_project_root()

/home/nirbaanm/workspace/git/nn_catalyst/src/mordred//checkpoints/stn_r3_f849_tlast29


'/home/nirbaanm/workspace/git/nn_catalyst/src/mordred'

In [5]:
import pickle
from sklearn.preprocessing import StandardScaler

# Define the path to the pickle file
pickle_file_path = f'{CHECKPOINTS_FOLDER}/scaler_X.pkl'

# Load the StandardScaler from the pickle file
with open(pickle_file_path, 'rb') as file:
    scaler = pickle.load(file)

# Now you can use the scaler
# Example: scaler.transform(data)

# Standardize features
X_scaled = scaler.transform(X)
X_tensor = torch.tensor(X_scaled, dtype=torch.float32)


FileNotFoundError: [Errno 2] No such file or directory: '/home/nirbaanm/workspace/git/nn_catalyst/src/mordred//checkpoints/stn_r3_f849_tlast29/scaler_X.pkl'

In [ ]:

# Load models and make predictions
predictions = []
for target_index in range(df.shape[1] - 849):
    # Initialize model
    model = SingleTargetNet(X.shape[1])
    
    # Load saved weights
    model.load_state_dict(torch.load(f'checkpoints/best_model_target_{target_index}.pth'))
    
    # Set to evaluation mode
    model.eval()
    
    # Make predictions
    with torch.no_grad():
        pred = model(X_tensor).numpy()
        predictions.append(pred)

# Stack predictions into array
predictions = np.hstack(predictions)

# Create a DataFrame for predictions
predictions_df = pd.DataFrame(predictions, columns=[f'Prediction_{i}' for i in range(predictions.shape[1])])

# Save predictions to a CSV file
predictions_df.to_csv('predictions.csv', index=False)

print(f"Predictions shape: {predictions.shape}")